In [1]:
import numpy as np
import os
import pandas as pd

In [2]:
from IPython.display import display, clear_output

import math
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from pathlib import Path




## Load Data

In [3]:
DATA_PATH = Path("data_original")

In [4]:
%%time
users_df = pd.read_csv(DATA_PATH / 'users.csv')
items_df = pd.read_csv(DATA_PATH / 'items.csv')
interactions_df = pd.read_csv(DATA_PATH / 'interactions.csv')
interactions_df = interactions_df[interactions_df['last_watch_dt'] < '2021-04-01']

CPU times: total: 1.64 s
Wall time: 2.26 s


In [5]:
interactions_df.shape

(263874, 5)

## Data Preparation

In [6]:
users_interactions_count_df = interactions_df.groupby(['user_id', 'item_id']).size().groupby('user_id').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['user_id']]
print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

# users: 86614
# users with at least 5 interactions: 14563


In [7]:
print('# of interactions: %d' % len(interactions_df))
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'user_id',
               right_on = 'user_id')
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions: 263874
# of interactions from users with at least 5 interactions: 142670


In [8]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['user_id', 'item_id'])['watched_pct'].sum() \
                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 142670


,user_id,item_id,watched_pct
0,21,849,6.375039
1,21,4345,6.658211
2,21,10283,6.658211
3,21,12261,6.658211
4,21,15997,6.658211
5,32,952,6.044394
6,32,4382,4.954196
7,32,4807,6.658211
8,32,10436,6.658211
9,32,12132,6.658211


In [9]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['user_id'], 
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 114136
# interactions on Test set: 28534


In [10]:

#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('user_id')
interactions_train_indexed_df = interactions_train_df.set_index('user_id')
interactions_test_indexed_df = interactions_test_df.set_index('user_id')

In [11]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]['item_id']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [12]:
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:


    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(articles_df['item_id'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[person_id]
        if type(interacted_values_testset['item_id']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['item_id'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['item_id'])])  
        interacted_items_count_testset = len(person_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id, 
                                               items_to_ignore=get_items_interacted(person_id, 
                                                                                    interactions_train_indexed_df), 
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=item_id%(2**32))

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['item_id'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['item_id'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, person_id in enumerate(tqdm(list(interactions_test_indexed_df.index.unique().values))):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)  
            person_metrics['user_id'] = person_id
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()    

In [14]:
total_df = pd.concat([interactions_train_df,interactions_test_indexed_df.reset_index()])
total_df['user_id'], users_keys = total_df.user_id.factorize()
total_df['item_id'], items_keys = total_df.item_id.factorize()

train_encoded = total_df.iloc[:len(interactions_train_df)].values
test_encoded = total_df.iloc[len(interactions_train_df):].values

In [15]:
from scipy.sparse import csr_matrix
shape = [int(total_df['user_id'].max()+1), int(total_df['item_id'].max()+1)]
X_train = csr_matrix((train_encoded[:, 2], (train_encoded[:, 0], train_encoded[:, 1])), shape=shape).toarray()
X_test = csr_matrix((test_encoded[:, 2], (test_encoded[:, 0], test_encoded[:, 1])), shape=shape).toarray()

In [16]:
# Initialize the DataObject, which must return an element (features vector x and target value y)
# for a given idx. This class must also have a length atribute
class UserOrientedDataset(Dataset):
    def __init__(self, X):
        super().__init__() # to initialize the parent class
        self.X = X.astype(np.float32)
        self.len = len(X)

    def __len__(self): # We use __func__ for implementing in-built python functions
        return self.len

    def __getitem__(self, index):
        return self.X[index]

In [17]:
# Initialize DataLoaders - objects, which sample instances from DataObject-s
train_dl = DataLoader(
    UserOrientedDataset(X_train),
    batch_size = BATCH_SIZE,
    shuffle = True
)

test_dl = DataLoader(
    UserOrientedDataset(X_test),
    batch_size = EVAL_BATCH_SIZE,
    shuffle = False
)

dls = {'train': train_dl, 'test': test_dl}

## Model
Значимо изменить архитектуру модели (2 балла)

In [13]:
# Constants
SEED = 42 # random seed for reproducibility
LR = 1e-3 # learning rate, controls the speed of the training
WEIGHT_DECAY = 0.01 # lambda for L2 reg. ()
NUM_EPOCHS = 200 # num training epochs (how many times each instance will be processed)
GAMMA = 0.9995 # learning rate scheduler parameter
BATCH_SIZE = 3000 # training batch size
EVAL_BATCH_SIZE = 3000 # evaluation batch size.
DEVICE = 'cuda' #'cuda' # device to make the calculations on

In [19]:
class Model(nn.Module):
    def __init__(self, in_and_out_features = 8287):
        super().__init__()
        self.in_and_out_features = in_and_out_features
        self.hidden_size = 1024
        
        #BEFORE changes
        # self.sequential = nn.Sequential( # NN architecure, where the modules modify the data sequentially
        #     nn.Linear(in_and_out_features, self.hidden_size), # Linear transformation
        #     nn.ReLU(), # Activation function
        #     nn.Linear(self.hidden_size, in_and_out_features) # Another Linear transformation
        # )

        self.encoder = nn.Sequential( 
        nn.Linear(in_and_out_features, self.hidden_size*4),
        nn.ReLU(), 
        nn.Linear(self.hidden_size*4, self.hidden_size*2),
        nn.ReLU(),
        nn.Linear(self.hidden_size*2, self.hidden_size*2), 
        nn.ReLU(), # Activation function
        nn.Linear(self.hidden_size*2, self.hidden_size), 
        nn.ReLU()
        )

        self.decoder = nn.Sequential( 
        nn.Linear(self.hidden_size, self.hidden_size*2),
        nn.ReLU(), 
        nn.Linear(self.hidden_size*2, self.hidden_size*2),
        nn.ReLU(),
        nn.Linear(self.hidden_size*2, self.hidden_size*4), 
        nn.ReLU(), # Activation function
        nn.Linear(self.hidden_size*4, in_and_out_features) 
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [20]:
torch.manual_seed(SEED) # Fix random seed to have reproducible weights of model layers

model = Model()
#model.to(DEVICE)

# Initialize GD method, which will update the weights of the model
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# Initialize learning rate scheduler, which will decrease LR according to some rule
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

def rmse_for_sparse(x_pred, x_true):
    mask = (x_true > 0)
    sq_diff = (x_pred * mask - x_true) ** 2
    mse = sq_diff.sum() / mask.sum()
    return mse ** (1/2)

In [21]:
# Training loop
metrics_dict = {
    "Epoch": [],
    "Train RMSE": [],
    "Test RMSE": [],
}

# Train loop
for epoch in range(NUM_EPOCHS):
    metrics_dict["Epoch"].append(epoch)
    for stage in ['train', 'test']:
        with torch.set_grad_enabled(stage == 'train'): # Whether to start building a graph for a backward pass
            if stage == 'train':
                model.train() # Enable some "special" layers (will speak about later)
            else:
                model.eval() # Disable some "special" layers (will speak about later)

            loss_at_stage = 0 
            for batch in dls[stage]:
                #batch = batch.to(DEVICE)
                x_pred = model(batch) # forward pass: model(x_batch) -> calls forward()
                loss = rmse_for_sparse(x_pred, batch) # ¡Important! y_pred is always the first arg
                if stage == "train":
                    loss.backward() # Calculate the gradients of all the parameters wrt loss
                    optimizer.step() # Update the parameters
                    scheduler.step()
                    optimizer.zero_grad() # Zero the saved gradient
                loss_at_stage += loss.item() * len(batch)
            rmse_at_stage = (loss_at_stage / len(dls[stage].dataset)) ** (1/2)
            metrics_dict[f"{stage.title()} RMSE"].append(rmse_at_stage)
            
    if (epoch == NUM_EPOCHS - 1) or epoch % 10 == 9:
        clear_output(wait=True)
        display(pd.DataFrame(metrics_dict))

,Epoch,Train RMSE,Test RMSE
0,0,2.121857,2.201029
1,1,2.141730,1.867906
2,2,1.992629,1.949605
3,3,1.797448,1.841078
4,4,1.577153,1.697863
...,...,...,...
195,195,0.699927,1.428902
196,196,0.689256,1.425949
197,197,0.678649,1.433828
198,198,0.674293,1.430693


In [22]:
with torch.no_grad():
    X_pred = model(torch.Tensor(X_test))
X_pred

tensor([[ 6.1816,  6.2775,  3.7436,  ..., -0.3453,  0.1727,  0.0915],
        [ 0.4917,  1.5275,  1.8224,  ...,  0.0239, -0.1217,  0.1208],
        [ 6.6389,  6.0322,  5.5957,  ..., -0.0224, -0.0774,  0.1126],
        ...,
        [ 1.0012,  1.6987,  1.4982,  ..., -0.0654,  0.0641,  0.0464],
        [ 4.3691,  4.4737,  3.7697,  ..., -0.1351,  0.0582,  0.1432],
        [ 6.5518,  6.5737,  4.4914,  ..., -0.2870,  0.1215, -0.0574]])

In [23]:
class AERecommender:
    
    MODEL_NAME = 'Autoencoder'
    
    def __init__(self, X_preds, X_train_and_val, X_test):

        self.X_preds = X_preds.cpu().detach().numpy()
        self.X_train_and_val = X_train_and_val
        self.X_test = X_test
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_select_idx, topn=10, verbose=False):
        user_preds = self.X_preds[user_id][items_to_select_idx]
        items_idx = items_to_select_idx[np.argsort(-user_preds)[:topn]]

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        return items_idx

    def evaluate(self, size=100):

        X_total = self.X_train_and_val + self.X_test

        true_5 = []
        true_10 = []

        for user_id in range(len(X_test)):
            non_zero = np.argwhere(self.X_test[user_id] > 0).ravel()
            all_nonzero = np.argwhere(X_total[user_id] > 0).ravel()
            select_from = np.setdiff1d(np.arange(X_total.shape[1]), all_nonzero)

            for non_zero_idx in non_zero:
                random_non_interacted_100_items = np.random.choice(select_from, size=20, replace=False)
                preds = self.recommend_items(user_id, np.append(random_non_interacted_100_items, non_zero_idx), topn=10)
                true_5.append(non_zero_idx in preds[:5])
                true_10.append(non_zero_idx in preds)

        return {"recall@5": np.mean(true_5), "recall@10": np.mean(true_10)}
    
ae_recommender_model = AERecommender(X_pred, X_train, X_train)

In [24]:
ae_global_metrics = ae_recommender_model.evaluate()
ae_global_metrics

{'recall@5': 0.2713584990297132, 'recall@10': 0.5613426038791969}

In [27]:
# Объединяем датасет целиком
kion_dataset = total_df.values
shape = [int(total_df["user_id"].max() + 1), int(total_df["item_id"].max() + 1)]
full_data = csr_matrix((kion_dataset[:, 2], (kion_dataset[:, 0], kion_dataset[:, 1])), shape=shape).toarray()

In [28]:
# Формируем предсказания для оффлайн сервиса
with torch.no_grad():
    recos_full = model(torch.Tensor(full_data))

In [33]:
# Функция рекомендаций как была у меня в онлайн сервисе
def recommend(user_id, n=10):
    filtered_data = np.argwhere(full_data[user_id] > 0).ravel()
    res = np.setdiff1d(np.arange(full_data.shape[1]), filtered_data)
    # доделаем предсказания
    not_watched = np.random.choice(res, size=20, replace=False)
    recos = recos_full[user_id][not_watched]
    recos_result = not_watched[np.argsort(-recos)[:n]]
    return recos_result

In [34]:
all_users_ids = interactions_full_indexed_df.index.unique().tolist()

In [53]:
offline_recos = {}
for i, user_id in enumerate(all_users_ids):
    recos = recommend(i)
    offline_recos.update({user_id: list(recos)})


In [54]:
offline_recos

{21: [6945, 6914, 7655, 2798, 1598, 3518, 7115, 4406, 7413, 7401],
 32: [5024, 3239, 5596, 3031, 3757, 6097, 5995, 6919, 4550, 6538],
 83: [5495, 6305, 4446, 3743, 1125, 5763, 7257, 3618, 7559, 4887],
 133: [2027, 6775, 3443, 3710, 4200, 2227, 6802, 5307, 2631, 800],
 163: [4873, 5689, 7166, 1259, 7586, 142, 2251, 992, 5264, 1922],
 243: [2510, 5899, 1634, 4087, 3149, 3309, 1456, 715, 4044, 5710],
 313: [3247, 5546, 6855, 6561, 797, 3788, 2703, 3432, 3115, 7354],
 408: [7460, 4686, 7552, 5594, 6655, 5156, 579, 3450, 3798, 1476],
 571: [78, 4476, 4075, 4450, 496, 4864, 6403, 4492, 4130, 2754],
 576: [5412, 6088, 6298, 2761, 6560, 4740, 5949, 7540, 4821, 108],
 654: [3768, 2680, 2191, 2896, 5141, 5737, 313, 743, 4953, 716],
 658: [2040, 6540, 232, 4883, 4274, 5030, 4106, 5013, 6498, 5221],
 745: [4571, 4941, 1304, 949, 6743, 221, 6290, 3806, 1822, 2254],
 799: [5051, 3462, 5096, 2957, 3165, 2843, 2337, 2490, 419, 358],
 827: [1991, 3750, 7290, 1770, 2137, 6901, 1597, 1138, 1918, 3180],
 

In [56]:
import pickle

In [57]:
with open('offline_autoencoder.pkl', 'wb') as f:
    pickle.dump(offline_recos, f)